# Туториал по использованию библиотеки CatBoost

В этом туториале показана основная функциональность библиотеки CatBoost с использованием датасета Amazon из соревнования на [Kaggle](https://www.kaggle.com).
Данные можно скачать [здесь](https://www.kaggle.com/c/amazon-employee-access-challenge/data) (для этого надо создать свой аккаунт на Kaggle)

# Чтение данных

In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
import catboost
from catboost import *

In [ ]:
train_df = pd.read_csv('amazon/train.csv')
train_df.head()

# Подготовка датасета

Выделение целевой переменной

In [ ]:
y = train_df.ACTION
X = train_df.drop('ACTION', axis=1)

Объявление категориальных факторов

In [ ]:
cat_features = list(range(0, X.shape[1]))
print(cat_features)

Посмотрим на классы в обучающей выборке

In [ ]:
print('Zero count = ' + str(len(y) - sum(y)) + ', One count = ' + str(sum(y)))

# Способы задания датасета

In [ ]:
import numpy as np
from catboost import Pool

pool1 = Pool(data=X, label=y, cat_features=cat_features)
pool2 = Pool(data='amazon/train.csv', delimiter=',', has_header=True, column_description='amazon/train.cd')

print('Dataset shape')
print('dataset 1:' + str(pool1.shape) + '\ndataset 2:' + str(pool2.shape))

print('\n')
print('Column names')
print('dataset 1: ')
print(pool1.get_feature_names()) 
print('\ndataset 2:')
print(pool2.get_feature_names())

# Выделение подвыборки для контроля качества

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=1234)

# Обучение модели

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    learning_rate=0.1
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent'
)
print('Model is fitted: ' + str(model.is_fitted_))
print('Model params:')
print(model.get_params())

# Stdout алгоритма

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=15,
    #metric_period=5,
    #logging_level='Info'
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
)

# Рандом

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=5,
    #random_seed=0,
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
)

In [ ]:
print(model.random_seed_)

# Вычисление метрик и графики

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=50,
    random_seed=63,
    learning_rate=0.5,
    # loss_function='CrossEntropy', # 'Logloss', # 'Logloss:border=0.5'
    custom_loss=['AUC', 'Accuracy']
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

## Сравнение нескольких моделей

In [ ]:
model1 = CatBoostClassifier(
    learning_rate=0.5,
    iterations=100,
    train_dir='learing_rate_0.5'
)

model2 = CatBoostClassifier(
    learning_rate=0.01,
    iterations=100,
    train_dir='learing_rate_0.01'
)
model1.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    verbose=False
)
model2.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    verbose=False
)

In [ ]:
from catboost import MetricVisualizer
MetricVisualizer(['learing_rate_0.01', 'learing_rate_0.5']).start()

# Лучшая итерация

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=100,
    random_seed=63,
    learning_rate=0.5,
    #use_best_model=False
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

In [ ]:
print('Tree count: ' + str(model.tree_count_))

In [ ]:
model.shrink(35)
print('Tree count after shrink: ' + str(model.tree_count_))

# Кросс-валидация

In [ ]:
from catboost import cv

params = {}
params['loss_function'] = 'Logloss'
params['iterations'] = 80
params['custom_loss'] = 'AUC'
params['random_seed'] = 63
params['learning_rate'] = 0.5

cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=cat_features),
    fold_count=5,
    inverted=False,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=False,
    verbose=False
)

In [ ]:
print(cv_data[0:4])

In [ ]:
best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

In [ ]:
cv_data = cv(
    params = params,
    pool = Pool(X, label=y, cat_features=cat_features),
    fold_count=5,
    inverted=False,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)

best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

# Детектор переобучения

In [ ]:
model_with_early_stop = CatBoostClassifier(
    iterations=200,
    random_seed=63,
    learning_rate=0.5,
    od_type='Iter',
    od_wait=20
)
model_with_early_stop.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

In [ ]:
print(model_with_early_stop.tree_count_)

In [ ]:
model_with_early_stop = CatBoostClassifier(
    eval_metric='AUC',
    iterations=200,
    random_seed=63,
    learning_rate=0.5,
    od_type='Iter',
    od_wait=20
)
model_with_early_stop.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    logging_level='Silent',
    plot=True
)

In [ ]:
print(model_with_early_stop.tree_count_)

## Снепшоты

In [ ]:
#!rm 'snapshot.bkp'
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=40,
    save_snapshot=True,
    snapshot_file='snapshot.bkp',
    random_seed=43
)
model.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    logging_level='Verbose'
)

# Предсказание формулы

In [ ]:
print(model.predict_proba(data=X_validation))

In [ ]:
print(model.predict(data=X_validation))

In [ ]:
raw_pred = model.predict(data=X_validation, prediction_type='RawFormulaVal')
print(raw_pred)

In [ ]:
import math
def sigmoid(x):
  return 1 / (1 + math.exp(-x))
probabilities = [sigmoid(x) for x in raw_pred]
print(np.array(probabilities))

# Staged predict

In [ ]:
predictions_gen = model.staged_predict_proba(data=X_validation, ntree_start=0, ntree_end=5, eval_period=1)
for iteration, predictions in enumerate(predictions_gen):
    print('Iteration ' + str(iteration) + ', predictions:')
    print(predictions)

# Вычисление метрик на новом датасете

In [ ]:
metrics = model.eval_metrics(data=pool1, metrics=['Logloss','AUC'], ntree_start=0, ntree_end=0, eval_period=1, plot=True)

In [ ]:
print('AUC values:')
print(np.array(metrics['AUC']))

# Важность факторов

Обучим модель на побольше итераций.

In [ ]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=300,
    max_ctr_complexity=1,
    random_seed=43
)
model.fit(
    X, y,
    cat_features=cat_features,
    verbose=50
)

In [ ]:
# Найдем самые важные факторы
importances = model.feature_importances_
names = pool1.get_feature_names()

named_importances = [[importances[i], names[i]] for i in range(len(names))]
print(np.array(sorted(named_importances, reverse=True)))

# Shap values

In [ ]:
shap_values = model.get_feature_importance(pool1, fstr_type='ShapValues')
print(shap_values.shape)

In [ ]:
test_objects = [X.iloc[0:1], X.iloc[91:92]]

for obj in test_objects:
    print('Probability of class 1 = {:.4f}'.format(model.predict_proba(obj)[0][1]))
    print('Formula raw prediction = {:.4f}'.format(model.predict(obj, prediction_type='RawFormulaVal')[0]))
    print('\n')

In [ ]:
import shap
shap.initjs()
shap.force_plot(shap_values[0,:], X.iloc[0,:])

In [ ]:
import shap
shap.initjs()
shap.force_plot(shap_values[91,:], X.iloc[91,:])

In [ ]:
shap.summary_plot(shap_values, X)

In [ ]:
X_small = X.iloc[0:200]
shap_small = shap_values[:200]
shap.force_plot(shap_small, X_small)

# Оценка факторов

In [ ]:
from catboost.eval.catboost_evaluation import *
learn_params = {'iterations': 20, # 2000
                'learning_rate': 0.5, # we set big learning_rate, because we have small #iterations
                'random_seed': 0,
                'logging_level': 'Silent',
                'loss_function' : 'Logloss',
                'boosting_type': 'Plain'}
evaluator = CatboostEvaluation('amazon/train.tsv',
                               fold_size=10000, # <= 50% of dataset
                               fold_count=20,
                               column_description='amazon/train.cd',
                               partition_random_seed=0,
                               #working_dir=... 
)
result = evaluator.eval_features(learn_config=learn_params,
                                 eval_metrics=['Logloss', 'Accuracy'],
                                 features_to_eval=[6, 7, 8])

In [ ]:
from catboost.eval.evaluation_result import *
logloss_result = result.get_metric_results('Logloss')
logloss_result.get_baseline_comparison(ScoreConfig(ScoreType.Rel))

In [ ]:
logloss_result.get_baseline_comparison(ScoreConfig(ScoreType.Abs))

# Сохранение модели

In [ ]:
my_best_model = CatBoostClassifier(iterations=10)
my_best_model.fit(
    X_train, y_train,
    eval_set=(X_validation, y_validation),
    cat_features=cat_features,
    verbose=False
)
my_best_model.save_model('catboost_model.bin')

In [ ]:
my_best_model.load_model('catboost_model.bin')
print(my_best_model.get_params())

# Подбор параметров

# Скорость

In [ ]:
from catboost import CatBoost
fast_model = CatBoostClassifier(
    random_seed=63,
    iterations=150,
    learning_rate=0.01,
    boosting_type='Plain',
    bootstrap_type='Bernoulli',
    subsample=0.5,
    one_hot_max_size=20,
    rsm=0.5,
    leaf_estimation_iterations=5,
    max_ctr_complexity=1)

fast_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    logging_level='Silent',
    plot=True
)

# Качество

In [ ]:
tunned_model = CatBoostClassifier(
    random_seed=63,
    iterations=1000,
    learning_rate=0.03,
    l2_leaf_reg=3,
    bagging_temperature=1,
    random_strength=1,
    one_hot_max_size=2,
    leaf_estimation_method='Newton',
)
tunned_model.fit(
    X_train, y_train,
    cat_features=cat_features,
    logging_level='Silent',
    eval_set=(X_validation, y_validation),
    plot=True
)

# Как обучить лучшую модель

In [ ]:
best_model = CatBoostClassifier(
    random_seed=63,
    iterations=int(tunned_model.tree_count_ * 1.2),
)
best_model.fit(
    X, y,
    cat_features=cat_features,
    verbose=100
)

# Применим модель к отложенной выборке

In [ ]:
test_df = pd.read_csv('amazon/test.csv')
X_test = test_df.drop('id', axis=1)
test_pool = Pool(data=X_test, cat_features=cat_features)
contest_predictions = best_model.predict_proba(test_pool)
print('Predictoins:')
print(contest_predictions)

# Подготовим датасет к сабмиту

In [ ]:
f = open('submit.csv', 'w')
f.write('Id,Action\n')
for idx in range(len(contest_predictions)):
    line = str(test_df['id'][idx]) + ',' + str(contest_predictions[idx][1]) + '\n'
    f.write(line)
f.close()

# Еще один секрет

In [ ]:
best_model_with_test = CatBoostClassifier(
    random_seed=63,
    iterations=int(tunned_model.tree_count_ * 1.2),
)
best_model_with_test.fit(
    X, y,
    cat_features=cat_features,
    verbose=100,
    eval_set=test_pool,
)

In [ ]:
print(best_model_with_test.tree_count_)

In [ ]:
contest_predictions_with_test = best_model_with_test.predict_proba(test_pool)
f = open('submit_with_test.csv', 'w')
f.write('Id,Action\n')
for idx in range(len(contest_predictions)):
    line = str(test_df['id'][idx]) + ',' + str(contest_predictions_with_test[idx][1]) + '\n'
    f.write(line)
f.close()

Удачи!!!